In [1]:
# Generate data for the Darcy flow problem
# Input: a - the permeability
# output: u - the flow through the permeability
# data generated:
# 5 datasets of 200 training pairs, with a resolution of 256
# 1 dataset of 200 testing pairs, with a resolution of 256


In [ ]:
pip install torch

In [ ]:
# tensorflow for Gaussian Random Fields function:

In [ ]:
pip install tensorflow

In [ ]:
pip install --user git+https://github.com/devitocodes/devito.git

In [1]:
import torch
import math
from devito import div, grad, Eq, Operator, TimeFunction, Function, solve, Grid, configuration
import numpy as np
from random_fields import GaussianRF
from timeit import default_timer
import scipy.io

2022-08-29 17:01:31.348429: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-29 17:01:31.348497: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
np.random.seed(42)

In [3]:
import os

In [4]:
os.environ['DEVITO_LANGUAGE'] = 'openmp'

os.environ['OMP_SCHEDULE'] = 'STATIC'

os.environ['OMP_PROC_BIND'] = 'true'

os.environ['OMP_PLACES'] = 'cores'

In [5]:
'''
function to generate 'u' from 'a' using Devito
parameters 
-----------------
perm: tensor of size (1, s, s)
    this is "a"
f: tensor of size (s, s)
    the forcing function f(x) = 1
s: int
    the resolution
 '''
def darcy_flow_2d(perm, f, s):
    
    # a(x) is the coefficients
    # f is the forcing function
    # give the a and f1 functions the same data as inputs perm and f
    for i in range(s):
        f1.data[i] = f[i]
        a.data[i] = perm[i]
    
    # call operator for the 15,000th psuedo-timestep
    op(time= 15000)
   
    return torch. from_numpy(u.data[0])

In [ ]:
# Silence the runtime performance logging
configuration['log-level'] = 'ERROR'

# this is used to generate data for training the FNO and initial test data used as a ground truth

# number of grid points on [0,1]^2 
s = 256

# Number of solutions to generate
N = 200

numberofbatches = [1,5]
for num in numberofbatches:
# number of batches to generate (=1 for testing data and =5 for training data)
    for b in range(num):

        # Set up 2d GRF with covariance parameters to generate random coefficients
        norm_a = GaussianRF(2, s, alpha=2, tau=3)

        # Forcing function, f(x) = 1 
        f = np.ones((s, s))

        # create s x s grid with spacing 1
        grid = Grid(shape=(s, s), extent=(1.0,1.0))

        # create function on grid
        # space order of 2 to enable 2nd derivative
        # TimeFunction for u can be used despite the lack of a time-dependence. This is done for psuedotime
        u = TimeFunction(name='u', grid=grid, space_order=2)
        a = Function(name='a', grid=grid, space_order=2)
        f1 = Function(name='f1', grid=grid, space_order=2)

        # define 2D Darcy flow equation
        equation_u = Eq(-a*u.laplace,f1)

        # Let SymPy solve for the central stencil point
        stencil = solve(equation_u, u)

        # Let our stencil populate the buffer `u.forward`
        update = Eq(u.forward, stencil)

        # Boundary Conds
        x, y = grid.dimensions
        t = grid.stepping_dim
        nx = s
        ny = s
        bc = [Eq(u[t+1, 0, y],u[t+1, 1,y])]  # du/dx = 0 for x=0.
        bc += [Eq(u[t+1,nx-1, y],u[t+1,nx-2, y])]  # du/dx = 0 for x=1.
        bc += [Eq(u[t+1, x, 0],u[t+1,x ,1])]  # du/dx = 0 at y=0
        bc += [Eq(u[t+1, x, ny-1],u[t+1, x, ny-2])]  # du/dx=0 for y=1
        # u=0 for all sides
        bc = [Eq(u[t+1, x, 0], 0.)]
        bc += [Eq(u[t+1, x, ny-1], 0.)]
        bc += [Eq(u[t+1, 0, y], 0.)]
        bc += [Eq(u[t+1, nx-1, y], 0.)]

        op = Operator([update] + bc)

        # all the inputs
        coeff = torch.zeros(N, s, s)
        #all the solutions
        sol = torch.zeros(N, s, s)

        c = 0
        t0 = default_timer()

        for j in range(N):

            #Sample random fields
            # create a threshold, either 4 or 12 (common for permeability)
            thresh_a = norm_a.sample(1)
            thresh_a[thresh_a>=0] = 12
            thresh_a[thresh_a<0] = 4

            w1 = thresh_a[0]

            #Solve df either for w0 (exp perm) or w1 (thresh perm)
            output = darcy_flow_2d(w1, f, s)


            coeff[c:(c+1),...] = w1
            sol[c:(c+1),...] = output

            c += 1
        
        if num == 5:
            scipy.io.savemat('df_train_{}_{}.mat'.format(s,b), mdict={'coeff': coeff.cpu().numpy(), 'sol': sol.cpu().numpy()})
        else:
            scipy.io.savemat('df_test_{}.mat'.format(s), mdict={'coeff': coeff.cpu().numpy(), 'sol': sol.cpu().numpy()})
    
    t1 = default_timer()
    print("number of samples:", num*200, "Time taken:", t1-t0)
       
